In [1]:
import pandas as pd
import numpy as np

In [4]:
# Import the database and stitch together the pages
steam = pd.read_json("https://steamspy.com/api.php?request=all&page=0", orient="index")
steam = steam.append(
    [pd.read_json("https://steamspy.com/api.php?request=all&page={}".format(i), orient="index") for i in range(1, 2)])

# Preview 'unclean' dataset
steam.head()


,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
570,570,Dota 2,Valve,Valve,,1364873,263943,0,"100,000,000 .. 200,000,000",33693,1497,799,674,0,0,0,671035
730,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,5335135,717424,0,"50,000,000 .. 100,000,000",28074,825,6490,292,0,0,0,949451
578080,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1029684,838306,0,"50,000,000 .. 100,000,000",24721,595,11417,250,2999,2999,0,353231
440,440,Team Fortress 2,Valve,Valve,,779844,52003,0,"50,000,000 .. 100,000,000",8064,1629,343,514,0,0,0,107161
304930,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,,419374,39279,0,"20,000,000 .. 50,000,000",7108,2203,333,321,0,0,0,28380


In [5]:
# Sort by concurrent users
steam = steam.sort_values(by="ccu", ascending=False)

# Remove duplicates
steam = steam.drop_duplicates()

# Remove columns that lacked useful values
steam = steam.drop(["score_rank", "userscore", "price"], axis=1)

# Rename some columns for clarity
steam = steam.rename({
    "positive": "positive_reviews",
    "negative": "negative_reviews",
    "initialprice": "price"
}, axis="columns")

# Add an index
steam.reset_index(drop=True, inplace=True)

In [6]:
# Modify the "owners" values to use dashes to look more readable

def f(row):
    return("-".join(row.split("..")))

steam["owners"] = steam["owners"].map(f)

In [7]:
# Modify the "price" to include decimal points

def p(price):

    price = str(price)
    return(float(".".join([price[:len(price) - 2], price[len(price) - 2:]])))


steam["price"] = steam["price"].map(p)


In [8]:
# Bring in another dataset for additional information

ks = pd.read_csv("kagglesteam.csv")

In [9]:
ks = ks[["appid", "release_date", "english", "categories", "genres"]]

left = steam[np.in1d(steam["appid"], ks["appid"].unique())].sort_values("appid")

In [10]:
right = ks[np.in1d(ks["appid"], left["appid"].unique())].sort_values("appid")

In [11]:
# Merge to obtain the final steam (fs) dataset
fs = pd.merge(left, right)



In [12]:
fs = fs[["appid", "name", "release_date", "developer", "publisher", "positive_reviews", "negative_reviews", "owners", "average_forever", "average_2weeks", "median_forever", "median_2weeks", "price", "ccu", "english", "categories", "genres"]]

# fs.head(50)
fs.shape


(1593, 17)

In [13]:
fs.to_csv("finalsteam.csv", index=False)
